# wis_dnn_challenge.py

In [1]:
import pandas as pd
import numpy as np
import os
import sys

# The time series that you would get are such that the difference between two rows is 15 minutes.
# This is a global number that we used to prepare the data, so you would need it for different purposes.
DATA_RESOLUTION_MIN = 15


class Predictor(object):
    """
    This is where you should implement your predictor.
    The testing script calls the 'predict' function with the glucose and meals test data which you will need in order to
    build your features for prediction.
    You should implement this function as you wish, just do not change the function's signature (name, parameters).
    The other functions are here just as an example for you to have something to start with, you may implement whatever
    you wish however you see fit.
    """

    def __init__(self, path2data=''):
        """
        This constructor only gets the path to a folder where the training data frames are.
        :param path2data: a folder with your training data.
        """
        self.path2data = path2data
        self.train_glucose = None
        self.train_meals = None
        self.nn = None

    def predict(self, X_glucose, X_meals):
        """
        You must not change the signature of this function!!!
        You are given two data frames: glucose values and meals.
        For every timestamp (t) in X_glucose for which you have at least 12 hours (48 points) of past glucose and two
        hours (8 points) of future glucose, predict the difference in glucose values for the next 8 time stamps
        (t+15, t+30, ..., t+120).

        :param X_glucose: A pandas data frame holding the glucose values in the format you trained on.
        :param X_meals: A pandas data frame holding the meals data in the format you trained on.
        :return: A numpy ndarray, sized (M x 8) holding your predictions for every valid row in X_glucose.
                 M is the number of valid rows in X_glucose (number of time stamps for which you have at least 12 hours
                 of past glucose values and 2 hours of future glucose values.
                 Every row in your final ndarray should correspond to:
                 (glucose[t+15min]-glucose[t], glucose[t+30min]-glucose[t], ..., glucose[t+120min]-glucose[t])
        """

        # build features for set of (ID, timestamp)
        X, y_true = self.build_features(X_glucose, X_meals)

        # feed the network you trained (this for example would be a horrible prediction)
        y = pd.concat([X.mean(1) for i in range(8)], axis=1)
        return y

    def define_nn(self):
        """
        Define your neural network.
        :return: None
        """
        self.nn = None
        return

    def train_nn(self, X_train, y_train):
        """
        Train your network using the training data.
        :param X_train: A pandas data frame holding the features
        :param y_train: A numpy ndarray, sized (M x 8) holding the values you need to predict.
        :return:
        """
        pass

    def save_nn_model(self):
        """
        Save your neural network after training.
        :return:
        """
        pass

    def load_nn_model(self):
        """
        Load your trained neural network.
        :return:
        """
        pass

    @staticmethod
    def load_data_frame(path):
        """
        Load a pandas data frame in the relevant format.
        :param path: path to csv.
        :return: the loaded data frame.
        """
        return pd.read_csv(path, index_col=[0, 1], parse_dates=['Date'])

    def load_raw_data(self):
        """
        Loads raw data frames from csv files, and do some basic cleaning
        :return:
        """
        self.train_glucose = Predictor.load_data_frame(os.path.join(self.path2data, 'GlucoseValues.csv'))
        self.train_meals = Predictor.load_data_frame(os.path.join(self.path2data, 'Meals.csv'))

        # suggested procedure
        # 1. handle outliers: trimming, clipping...
        # 2. feature normalizations
        # 3. resample meals data to match glucose values intervals
        return

    def build_features(self, X_glucose, X_meals, n_previous_time_points=48, n_future_time_points=8):
        """
        Given glucose and meals data, build the features needed for prediction.
        :param X_glucose: A pandas data frame holding the glucose values.
        :param X_meals: A pandas data frame holding the meals data.
        :param n_previous_time_points:
        :param n_future_time_points:
        :return: The features needed for your prediction, and optionally also the relevant y arrays for training.
        """
        # using X_glucose and X_meals to build the features
        # get the past 48 time points of the glucose
        X = X_glucose.reset_index().groupby('id').apply(Predictor.create_shifts, n_previous_time_points=n_previous_time_points).set_index(['id', 'Date'])
        # use the meals data...
        
        # this implementation of extracting y is a valid one.
        y = X_glucose.reset_index().groupby('id').apply(Predictor.extract_y, n_future_time_points=n_future_time_points).set_index(['id', 'Date'])
        index_intersection = X.index.intersection(y.index)
        X = X.loc[index_intersection]
        y = y.loc[index_intersection]
        return X, y

    @staticmethod
    def create_shifts(df, n_previous_time_points=48):
        """
        Creating a data frame with columns corresponding to previous time points
        :param df: A pandas data frame
        :param n_previous_time_points: number of previous time points to shift
        :return:
        """
        for g, i in zip(range(DATA_RESOLUTION_MIN, DATA_RESOLUTION_MIN*(n_previous_time_points+1), DATA_RESOLUTION_MIN),
                        range(1, (n_previous_time_points+1), 1)):
            df['GlucoseValue -%0.1dmin' % g] = df.GlucoseValue.shift(i)
        return df.dropna(how='any', axis=0)

    @staticmethod
    def extract_y(df, n_future_time_points=8):
        """
        Extracting the m next time points (difference from time zero)
        :param n_future_time_points: number of future time points
        :return:
        """
        for g, i in zip(range(DATA_RESOLUTION_MIN, DATA_RESOLUTION_MIN*(n_future_time_points+1), DATA_RESOLUTION_MIN),
                        range(1, (n_future_time_points+1), 1)):
            df['Glucose difference +%0.1dmin' % g] = df.GlucoseValue.shift(-i) - df.GlucoseValue
        return df.dropna(how='any', axis=0).drop('GlucoseValue', axis=1)

# Evaluation code

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

def compute_mean_pearson(y_true, y_pred, individual_index_name='id', n_future_time_points=8):
    """
    This function takes the true glucose values and the predicted ones, flattens the data per individual and then
    computed the Pearson correlation between the two vectors per individual.
    
    **This is how we will evaluate your predictions, you may use this function in your code**
    
    :param y_true: an M by n_future_time_points data frame holding the true glucose values
    :param y_pred: an M by n_future_time_points data frame holding the predicted glucose values
    :param individual_index_name: the name of the individual's indeces, default is 'id'
    :param n_future_time_points: number of future time points to predict, default is 8
    :return: the mean Pearson correlation
    """
    # making sure y_true and y_pred are of the same size
    assert y_true.shape == y_pred.shape
    # making sure y_true and y_pred share the same exact indeces and index names
    assert (y_true.index == y_pred.index).all() and y_true.index.names == y_pred.index.names
    # making sure that individual_index_name is a part of the index of both dataframes
    assert individual_index_name in y_true.index.names and individual_index_name in y_pred.index.names
    
    # concat data frames
    joined_df = pd.concat((y_true, y_pred), axis=1)
    return joined_df.groupby(individual_index_name)\
                    .apply(lambda x: pearsonr(x.iloc[:, :n_future_time_points].values.ravel(), 
                                              x.iloc[:, n_future_time_points:].values.ravel())[0]).mean()

# How to extract y_true

In this example, we demonstrate how to obtain the y_true that we will use for evaluation

In [4]:
# creating a Predictor instance
predictor = Predictor()

In [13]:
# load the GlucoseValues that you got for training
X_glucose = Predictor.load_data_frame('GlucoseValues.csv')

In [14]:
# run "build_features" to extract y_true
X, y_true = predictor.build_features(X_glucose, None)

In [15]:
# how X looks like
X.head()

GlucoseValue  GlucoseValue -15min  \
id Date                                                     
46 2015-01-05 22:18:00         108.0                100.8   
   2015-01-05 22:33:00         115.2                108.0   
   2015-01-05 22:48:00         120.6                115.2   
   2015-01-05 23:03:00         120.6                120.6   
   2015-01-05 23:18:00         124.2                120.6   

                        GlucoseValue -30min  GlucoseValue -45min  \
id Date                                                            
46 2015-01-05 22:18:00                104.4                115.2   
   2015-01-05 22:33:00                100.8                104.4   
   2015-01-05 22:48:00                108.0                100.8   
   2015-01-05 23:03:00                115.2                108.0   
   2015-01-05 23:18:00                120.6                115.2   

                        GlucoseValue -60min  GlucoseValue -75min  \
id Date                                                            
46 2015-01-05 22:18:00                122.4                127.8   
   2015-01-05 22:33:00                115.2                122.4   
   2015-01-05 22:48:00                104.4                115.2   
   2015-01-05 23:03:00                100.8                104.4   
   2015-01-05 23:18:00                108.0                100.8   

                        GlucoseValue -90min  GlucoseValue -105min  \
id Date                                                             
46 2015-01-05 22:18:00                122.4                 111.6   
   2015-01-05 22:33:00                127.8                 122.4   
   2015-01-05 22:48:00                122.4                 127.8   
   2015-01-05 23:03:00                115.2                 122.4   
   2015-01-05 23:18:00                104.4                 115.2   

                        GlucoseValue -120min  GlucoseValue -135min  \
id Date                                                              
46 2015-01-05 22:18:00                 109.8                 115.2   
   2015-01-05 22:33:00                 111.6                 109.8   
   2015-01-05 22:48:00                 122.4                 111.6   
   2015-01-05 23:03:00                 127.8                 122.4   
   2015-01-05 23:18:00                 122.4                 127.8   

                                ...           GlucoseValue -585min  \
id Date                         ...                                  
46 2015-01-05 22:18:00          ...                          192.6   
   2015-01-05 22:33:00          ...                          230.4   
   2015-01-05 22:48:00          ...                          235.8   
   2015-01-05 23:03:00          ...                          223.2   
   2015-01-05 23:18:00          ...                          205.2   

                        GlucoseValue -600min  GlucoseValue -615min  \
id Date                                                              
46 2015-01-05 22:18:00                 131.4                 111.6   
   2015-01-05 22:33:00                 192.6                 131.4   
   2015-01-05 22:48:00                 230.4                 192.6   
   2015-01-05 23:03:00                 235.8                 230.4   
   2015-01-05 23:18:00                 223.2                 235.8   

                        GlucoseValue -630min  GlucoseValue -645min  \
id Date                                                              
46 2015-01-05 22:18:00                 118.8                 124.2   
   2015-01-05 22:33:00                 111.6                 118.8   
   2015-01-05 22:48:00                 131.4                 111.6   
   2015-01-05 23:03:00                 192.6                 131.4   
   2015-01-05 23:18:00                 230.4                 192.6   

                        GlucoseValue -660min  GlucoseValue -675min  \
id Date                                                              
46 2015-01-05 22:18:00                 122.4          

In [16]:
# how y_true looks like
y_true.head()

Glucose difference +15min  Glucose difference +30min  \
id Date                                                                        
46 2015-01-05 22:18:00                        7.2                       12.6   
   2015-01-05 22:33:00                        5.4                        5.4   
   2015-01-05 22:48:00                        0.0                        3.6   
   2015-01-05 23:03:00                        3.6                       10.8   
   2015-01-05 23:18:00                        7.2                        1.8   

                        Glucose difference +45min  Glucose difference +60min  \
id Date                                                                        
46 2015-01-05 22:18:00                       12.6                       16.2   
   2015-01-05 22:33:00                        9.0                       16.2   
   2015-01-05 22:48:00                       10.8                        5.4   
   2015-01-05 23:03:00                        5.4                       -5.4   
   2015-01-05 23:18:00                       -9.0                       -9.0   

                        Glucose difference +75min  Glucose difference +90min  \
id Date                                                                        
46 2015-01-05 22:18:00                       23.4                       18.0   
   2015-01-05 22:33:00                       10.8                        0.0   
   2015-01-05 22:48:00                       -5.4                       -5.4   
   2015-01-05 23:03:00                       -5.4                       -1.8   
   2015-01-05 23:18:00                       -5.4                       -7.2   

                        Glucose difference +105min  Glucose difference +120min  
id Date                                                                         
46 2015-01-05 22:18:00                         7.2                         7.2  
   2015-01-05 22:33:00                         0.0                         3.6  
   2015-01-05 22:48:00                        -1.8                        -3.6  
   2015-01-05 23:03:00                        -3.6                        -3.6  
   2015-01-05 23:18:00                        -7.2                        -5.4

**The indexes of your predictions should be identical to that of y_true**

In [18]:
print ('shape of X:', X.shape)
print ('shape of y_true:', y_true.shape)

shape of X: (822001, 49)
shape of y_true: (822001, 8)


# Get y_pred

When we will run the test, here you will get a new glucose and meals data frames

If you do any preprocessing to your X before feeding it to the model, this should be done from within the predict function, as this is the only function that we will call.

In [19]:
y_pred = predictor.predict(X_glucose, None)

**Note that the indexes of y_true and y_pred are exactly the same**

In [26]:
assert (y_true.index == y_pred.index).all() and y_true.index.names == y_pred.index.names

# Evaluate

In [27]:
compute_mean_pearson(y_true, y_pred)

-0.1463717115200436